In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
Prepare consolidated data

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json

print(__doc__)


class Consolidate:
    
    def __init__(self):
        pass
        
    def read(self, file):
        data = pd.read_excel(file)
        return data

    def clean_data(self, data):
    
        # Select numeric data alone from data object
        numeric_data = data.loc[:, data.dtypes != object]
    
        # Select string data alone from data object to trim
        trim_data = data.select_dtypes(['object'])
    
        # Trim all string objects
        trim_data = trim_data.apply(lambda x: x.str.strip())
    
        # Merge numeric and string data in to data object
        data = pd.concat([numeric_data, trim_data], axis=1)
    
        # Change column name to lower case
        data.columns = [x.lower() for x in data.columns]
        return data
    
    def load_data(self, data):
            
        # Split dataframe into key and records
        key = data['cust_id']
    
        record_json = data.to_dict(orient='records')
    
        config = {
            'hosts': [ ('127.0.0.1', 3000) ]
        }

        # Create a client and connect it to the cluster
        try:
            client = aerospike.client(config).connect()
        except:
            import sys
            print("failed to connect to the cluster with", config['hosts'])
            sys.exit(1)
    
        # Records are addressable via a tuple of (namespace, set, key)
        for pk, bin in zip(key, record_json):
            string = "('dev', 's1bd', "
            string +=`pk`
            string = string + ")"
        
            from ast import literal_eval as make_tuple
            string = make_tuple(string)
        
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.put(string, bin)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)

            # Read a record after insert for verification
            (key, metadata, record) = client.get(string)
            print (key, record)
        
            # Close the connection to the Aerospike cluster
            client.close()
    
    def remove_data(self, data):

        key = data['cust_id']
    
        config = {
          'hosts': [ ('127.0.0.1', 3000) ]
        }

        # Create a client and connect it to the cluster
        try:
            client = aerospike.client(config).connect()
        except:
            import sys
            print("failed to connect to the cluster with", config['hosts'])
            sys.exit(1)
    
        # Count a record for verification before delete
        #query = client.query('test', 'customer')
        #query.select('frequency' )
        #query.apply('countNumOfBinsWithValues', 'count')

        # Callback function prints the records as they are read
        #def print_result(value):
        #    print(value)

        # Execute the query and call print_result for each result
        #query.foreach(print_result)
        
        # Records are addressable via a tuple of (namespace, set, key)
        for pk in key:
            string = "('dev', 's1bd', "
            string +=`pk`
            string = string + ")"
        
            from ast import literal_eval as make_tuple
            string = make_tuple(string)
        
            try:
                client = aerospike.client(config).connect()
                # Remove a record
                client.remove(string)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)

            # Count a record for verification after delete
    
            # Close the connection to the Aerospike cluster
            client.close()
        
def drive(mode):
    
    cons = Consolidate()

    # Read customer details excel
    data1 = cons.read("/home/azureuser/Aerospike/data/customer/customer details.xls")
    
    # Clean data1
    data1 = cons.clean_data(data1)
       
    # Read Customer vehicle excel
    data3 = cons.read("/home/azureuser/Aerospike/data/customer/Customer_vehicle.xls")
    
    # Clean data3
    data3 = cons.clean_data(data3)
    
    # Merge data1 and data3 by cust_id
    data3 = pd.merge(data1, data3, on='cust_id', how='inner')
    
    # Read Motor vehicle report excel
    data4 = cons.read("/home/azureuser/Aerospike/data/customer/Motor_vehicle_report.xls")
    
    # Clean data4
    data4 = cons.clean_data(data4)
    
    # Merge data3 and data4 by vehicle_no
    data4 = pd.merge(data3, data4, on='vehicle_no', how='inner')
    
    if mode == 'load':
        # Load data in to aerospike
        cons.load_data(data4)
    else:
        # Remove data from aerospike
        cons.remove_data(data4)
    
if __name__ == '__main__':
    drive('load')


Prepare consolidated data




error: (21L, 'A bin name should not exceed 14 characters limit', 'src/main/conversions.c', 506)


RecordNotFound: (2L, 'AEROSPIKE_ERR_RECORD_NOT_FOUND', 'src/main/client/get.c', 113)